
String equations
string unification
word problem
uniform word problem

https://skos.ii.uni.wroc.pl/pluginfile.php/73431/mod_resource/content/9/notes.pdf notes on word equations solving

https://youtu.be/MLwZrU6dOwE?si=TuKdKXKCj65xsYXS https://icetcs.github.io/frocos-itp-tableaux25/slides/tblfro/tableaux7-eisenhofer.pdf https://doi.org/10.1007/978-3-032-06085-3_7 On Solving String Equations via Powers and Parikh Images
https://icetcs.github.io/frocos-itp-tableaux25/recordings.html 

"
- Automatons (z3-noodler, Ostrich, Sloth, TRAU, . . . )
- SAT (Woorpje, nfa2sat)
- Unwinding/model repair (z3, cvc5, S3, z3Str3, . . . )
- Nielsen transformation (more or less in all solvers)
Termination only on “common-fragments”
quadratic, 2-variable, straight-line, chain-free
"

Nielson - from free grou thoery. Non temrimating in general https://en.wikipedia.org/wiki/Nielsen_transformation . Combinatorial group theory. 
parikh ~ character count. Can show some eqwuations can't be solved

Even if nielson happens inside, I could do it outseide.



word constraints
string constraints

word equations https://en.wikipedia.org/wiki/Word_equation
https://en.wikipedia.org/wiki/Combinatorics_on_words

https://drops.dagstuhl.de/storage/00lipics/lipics-vol152-csl2020/LIPIcs.CSL.2020.3/LIPIcs.CSL.2020.3.pdf solving word equatioms by recompressions 2020
also works on traces (partially commutative monoids?)
bubble sort for canonization.

https://github.com/remora-lang/remora/blob/main/src/Symbolic.hs#L65-L88

https://github.com/jrslepak/makanin-algo


https://homepage.divms.uiowa.edu/~ajreynol/pres-tpss2021.pdf cvc5 presentation summary of string solvers

https://assets.amazon.science/be/e6/c0265a14477ca4558784008404ea/solving-string-constraints-using-sat.pdf solving strings using sat 2023 aws

z3-noodler

cvc5

ostrich2

libmata


necklaces and rings.
cycle rewriting. zantema https://arxiv.org/abs/1609.07065 Termination of Cycle Rewriting by Transformation and Matrix Interpretation
https://cofault.com/aodt.html#aodt
Bag(Ring(x)) = e^Ring(x) = List(x) = 1/(1-x)
https://en.wikipedia.org/wiki/Monoid_factorisation#Chen%E2%80%93Fox%E2%80%93Lyndon_theorem   Chen–Fox–Lyndon theorem
Lyndon decomposition of monoid
https://en.wikipedia.org/wiki/Lyndon_word
A necklace as something else in the list, set, multiset hierarchy. You can't really associated

Nielson steps

```
au = bv
------
⊥

au = ε
------
⊥

wu = wv
------
u = v

xu = ε
-------
(xu = ε)[x/ε]

xu = av
------
(xu = av)[x/ε] (xu = av)[x/ax′]

xu = yv
------
(xu = yv)[x/y] (xu = yv)[x/yx′] (xu = yv)[y/xy′]
```

https://www.ccs.neu.edu/home/pete/pub/fmcad-2021.pdf Mathematical Programming Modulo Strings. Pete 2021

In [1]:
from dataclasses import dataclass
from typing import NamedTuple

class Var(NamedTuple):
    name : str

def subst(xs : list[object | Var], n : Var, s):
    res = []
    for x in xs:
        if n == x:
            res.extend(s)
        else:
            res.append(x)
    return res

counter = 0
def freshvar():
    global counter
    counter += 1
    return Var(f"_{counter}")


def subst_pair(pair : tuple[list[object | Var], list[object | Var]], n : Var, s):
    x,y = pair
    return (subst(x, n, s), subst(y, n, s))
from collections import deque

def nielson(x : list[object | Var], y : list[object | Var]):
    # mayb apply the nielson rules to front and back as is useful
    q = deque()
    subst = {}
    q.append((subst, ((x,y))))
    seen = set()
    n = 0
    while q:
        n += 1
        if n > 10:
            return
        (subst, (x,y)) = q.popleft()
        print("Nielson step:", x, y, subst)
        while x and y:
            a,b = x.pop(), y.pop()
            if a == b:
                continue
            else:
                # not equal. Do the junk
                # subst_pair copies btw
                x.append(a)
                y.append(b)
                p = (x,y)
                if not isinstance(a, Var) and not isinstance(b, Var):
                   pass # fail
                elif isinstance(a, Var) and isinstance(b, Var):
                    # both vars
                    z = freshvar()
                    q.append(({a : [b], **subst}, subst_pair(p, a, b ))) # assume equal
                    q.append(({b : [z,a] , **subst}, subst_pair(p, b, [z,a]) )) # a is prefix of b
                    q.append(({a : [z,b], **subst}, subst_pair(p, a, [z,b]) )) # b is prefix of a
                else:
                    if isinstance(b, Var):
                        a,b = b,a
                    # a is var, b is not
                    q.append(({a : [], **subst}, subst_pair((x,y), a, []))) # a is empty
                    #q.append(({a : [b], **subst}, subst_pair((x,y), a, [b]))) # a is empty
                    z = freshvar()
                    q.append(({a : [z,b], **subst}, subst_pair((x,y), a, [z,b]))) # b is prefix of a
                break
        else:
            if len(x) == 0 and len(y) == 0:
                yield subst
            elif len(y) == 0:
                if all(isinstance(v, Var) for v in x):
                    yield {**subst, **{v : [] for v in x}}
            elif len(x) == 0:
                if all(isinstance(v, Var) for v in y):
                    yield {**subst, **{v : [] for v in y}}

def canon_subst(subst, x):
    res = []
    for a in x:
        if isinstance(a, Var) and a in subst:
            subst[a] = Var(len(subst))
        else:
            res.append(a)
    return tuple(res)

def canon(p):
    x,y = p
    subst = {}
    x1 = canon_subst(subst, x)
    y1 = canon_subst(subst, y)
    return (x1,y1)


def compress(subst):
    res = {}
    for k,v in subst.items():
        s = []
        i = 0
        todo = list(reversed(v))
        while todo:
            x = todo.pop()
            if isinstance(x, Var) and x in subst:
                todo.extend(subst[x])
            else:
                s.append(x)
        res[k] = s
    return res

def cnielson(x : list[object | Var], y : list[object | Var]):
    for subst in nielson(x,y):
        yield {k:v for k,v in compress(subst).items() if not k.name.startswith("_")}

x,y,z = map(Var, "xyz")
assert list(cnielson([1,2], [1,2])) == [{}]
assert list(cnielson([1,2], [1,2,3])) == []
list(cnielson([x,2], [1,2]))
#assert list(nielson([x,2], [1,x])) == [] # doesn't terminate
assert list(cnielson([x,2], [1,2,3])) == []
list(cnielson([x,y], [1,2]))
list(cnielson([x,x], [1,2]))
list(cnielson([x,2], [2,x]))
#list(nielson([1,x], [1,2,y]))

Nielson step: [1, 2] [1, 2] {}
Nielson step: [1, 2] [1, 2, 3] {}
Nielson step: [Var(name='x'), 2] [1, 2] {}
Nielson step: [] [1] {Var(name='x'): []}
Nielson step: [Var(name='_1'), 1] [1] {Var(name='x'): [Var(name='_1'), 1]}
Nielson step: [Var(name='x'), 2] [1, 2, 3] {}
Nielson step: [Var(name='x'), Var(name='y')] [1, 2] {}
Nielson step: [Var(name='x')] [1, 2] {Var(name='y'): []}
Nielson step: [Var(name='x'), Var(name='_2'), 2] [1, 2] {Var(name='y'): [Var(name='_2'), 2]}
Nielson step: [] [1, 2] {Var(name='x'): [], Var(name='y'): []}
Nielson step: [Var(name='_3'), 2] [1, 2] {Var(name='x'): [Var(name='_3'), 2], Var(name='y'): []}
Nielson step: [Var(name='x')] [1] {Var(name='_2'): [], Var(name='y'): [Var(name='_2'), 2]}
Nielson step: [Var(name='x'), Var(name='_4'), 1] [1] {Var(name='_2'): [Var(name='_4'), 1], Var(name='y'): [Var(name='_2'), 2]}
Nielson step: [] [1] {Var(name='_3'): [], Var(name='x'): [Var(name='_3'), 2], Var(name='y'): []}
Nielson step: [Var(name='_5'), 1] [1] {Var(name='_

[{Var(name='x'): []},
 {Var(name='x'): [2]},
 {Var(name='x'): [2, 2]},
 {Var(name='x'): [2, 2, 2]},
 {Var(name='x'): [2, 2, 2, 2]}]